In [ ]:
#!pip install tsfresh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [tsfresh]


In [218]:
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
import plotly.express as px
from typing import Tuple
from tsfresh.feature_extraction import extract_features, EfficientFCParameters
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import optuna
pd.set_option('display.max_columns', None)

## Подготовка обучающих и валидационных временных рядов 

In [5]:
data = pd.read_csv('time_series_for_fitting.csv', index_col='Unnamed: 0')

In [102]:
data = pd.get_dummies(
    data,
    columns=["coal_name"],
    prefix="coal",        
    prefix_sep="_",       
    dtype=int,            
)

print(data.filter(like="coal_").head())

   coal_A1  coal_B2  coal_C3  coal_E5  coal_F6  coal_G7  coal_I9  coal_K10  \
0        0        0        0        1        0        0        0         0   
1        0        0        0        1        0        0        0         0   
2        0        0        0        1        0        0        0         0   
3        0        0        0        1        0        0        0         0   
4        0        0        0        1        0        0        0         0   

   coal_L11  
0         0  
1         0  
2         0  
3         0  
4         0  


In [149]:
data = data.fillna(0)

In [150]:
data.head()

,depot,stack,date,daily_input,daily_output,daily_delta,accum_weight,stack_temp_mean,temp_air_min,temp_air_max,temp_air_mean,humidity_min,humidity_max,humidity_mean,precipitation_min,precipitation_max,precipitation_mean,stack_id,fire_flag,month,rbf_month_01,rbf_month_02,rbf_month_03,rbf_month_04,rbf_month_05,rbf_month_06,rbf_month_07,rbf_month_08,rbf_month_09,rbf_month_10,rbf_month_11,rbf_month_12,coal_A1,coal_B2,coal_C3,coal_E5,coal_F6,coal_G7,coal_I9,coal_K10,coal_L11
0,3,1,2019-01-02,11984.1925,11984.1925,0.0,0.0,46.317536,4.4,9.4,7.529167,57.0,91.0,75.541667,0.0,0.9,0.216667,3_1,0,1.0,1.0,0.882497,0.606531,0.324652,0.135335,0.043937,0.011109,0.002187,0.000335,0.00004,0.000004,2.699579e-07,0,0,0,1,0,0,0,0,0
1,3,1,2019-01-06,11427.7060,11427.7060,0.0,0.0,57.904376,2.3,5.4,3.558333,83.0,93.0,89.583333,0.0,0.9,0.162500,3_1,0,1.0,1.0,0.882497,0.606531,0.324652,0.135335,0.043937,0.011109,0.002187,0.000335,0.00004,0.000004,2.699579e-07,0,0,0,1,0,0,0,0,0
2,3,1,2019-01-07,11984.1925,11984.1925,0.0,0.0,46.058287,0.9,3.9,1.816667,73.0,98.0,85.000000,0.0,3.0,0.604167,3_1,0,1.0,1.0,0.882497,0.606531,0.324652,0.135335,0.043937,0.011109,0.002187,0.000335,0.00004,0.000004,2.699579e-07,0,0,0,1,0,0,0,0,0
3,3,1,2019-01-10,25899.2620,25899.2620,0.0,0.0,36.533626,6.1,8.8,7.579167,51.0,75.0,63.375000,0.0,1.2,0.216667,3_1,0,1.0,1.0,0.882497,0.606531,0.324652,0.135335,0.043937,0.011109,0.002187,0.000335,0.00004,0.000004,2.699579e-07,0,0,0,1,0,0,0,0,0
4,3,1,2019-01-13,14248.8560,14248.8560,0.0,0.0,39.067186,0.6,4.5,1.779167,70.0,91.0,75.458333,0.0,0.1,0.004167,3_1,0,1.0,1.0,0.882497,0.606531,0.324652,0.135335,0.043937,0.011109,0.002187,0.000335,0.00004,0.000004,2.699579e-07,0,0,0,1,0,0,0,0,0


In [151]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4359 entries, 0 to 4358
Data columns (total 41 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   depot               4359 non-null   int64         
 1   stack               4359 non-null   int64         
 2   date                4359 non-null   datetime64[ns]
 3   daily_input         4359 non-null   float64       
 4   daily_output        4359 non-null   float64       
 5   daily_delta         4359 non-null   float64       
 6   accum_weight        4359 non-null   float64       
 7   stack_temp_mean     4359 non-null   float64       
 8   temp_air_min        4359 non-null   float64       
 9   temp_air_max        4359 non-null   float64       
 10  temp_air_mean       4359 non-null   float64       
 11  humidity_min        4359 non-null   float64       
 12  humidity_max        4359 non-null   float64       
 13  humidity_mean       4359 non-null   float64       
 1

In [152]:
data['date'] = pd.to_datetime(data['date'])

In [153]:
data.head(1)

,depot,stack,date,daily_input,daily_output,daily_delta,accum_weight,stack_temp_mean,temp_air_min,temp_air_max,temp_air_mean,humidity_min,humidity_max,humidity_mean,precipitation_min,precipitation_max,precipitation_mean,stack_id,fire_flag,month,rbf_month_01,rbf_month_02,rbf_month_03,rbf_month_04,rbf_month_05,rbf_month_06,rbf_month_07,rbf_month_08,rbf_month_09,rbf_month_10,rbf_month_11,rbf_month_12,coal_A1,coal_B2,coal_C3,coal_E5,coal_F6,coal_G7,coal_I9,coal_K10,coal_L11
0,3,1,2019-01-02,11984.1925,11984.1925,0.0,0.0,46.317536,4.4,9.4,7.529167,57.0,91.0,75.541667,0.0,0.9,0.216667,3_1,0,1.0,1.0,0.882497,0.606531,0.324652,0.135335,0.043937,0.011109,0.002187,0.000335,0.00004,0.000004,2.699579e-07,0,0,0,1,0,0,0,0,0


In [154]:
data.shape

(4359, 41)

In [155]:
def cut_time_series_frames(
    df,
    history_len,
    overlap=0.5,
    group_col="stack_id",
    time_col="date",
    max_gap_days=1,
):
    """
    df          : исходный датафрейм
    history_len : N, длина истории (длина окна = N+1)
    overlap     : доля перекрытия окон [0, 1)
    group_col   : идентификатор временного ряда (stack_id)
    time_col    : имя столбца с датой
    max_gap_days: допустимый разрыв (в днях) внутри одного сегмента
    """

    if not 0 <= overlap < 1:
        raise ValueError("overlap должно быть в диапазоне [0, 1).")

    frame_len = history_len + 1
    step = int(round(frame_len * (1 - overlap)))
    if step <= 0:
        step = 1

    df_sorted = df.sort_values([group_col, time_col]).copy()

    all_frames = []
    window_id = 0

    for stack_id, g in df_sorted.groupby(group_col):
        g = g.sort_values(time_col).reset_index(drop=False)
        g.rename(columns={"index": "orig_index"}, inplace=True)

        gaps = g[time_col].diff().dt.days
        break_idx = gaps[gaps > max_gap_days].index.tolist()
        bounds = [0] + break_idx + [len(g)]

        for start_seg, end_seg in zip(bounds[:-1], bounds[1:]):
            seg = g.iloc[start_seg:end_seg]
            length = len(seg)
            if length < frame_len:
                continue

            start_pos = 0
            while start_pos + frame_len <= length:
                frame = seg.iloc[start_pos:start_pos + frame_len].copy()
                frame["window_id"] = window_id
                frame["pos_in_window"] = np.arange(frame_len)
                all_frames.append(frame)

                window_id += 1
                start_pos += step

    if not all_frames:
        return pd.DataFrame()

    result = pd.concat(all_frames, ignore_index=True)
    return result

In [156]:
frames = cut_time_series_frames(
    data,
    history_len=5,    # + 1 день на прогноз 
    overlap=0.5,
    group_col="stack_id",
    time_col="date",
    max_gap_days=1,
)


In [157]:
frames.head()


,orig_index,depot,stack,date,daily_input,daily_output,daily_delta,accum_weight,stack_temp_mean,temp_air_min,temp_air_max,temp_air_mean,humidity_min,humidity_max,humidity_mean,precipitation_min,precipitation_max,precipitation_mean,stack_id,fire_flag,month,rbf_month_01,rbf_month_02,rbf_month_03,rbf_month_04,rbf_month_05,rbf_month_06,rbf_month_07,rbf_month_08,rbf_month_09,rbf_month_10,rbf_month_11,rbf_month_12,coal_A1,coal_B2,coal_C3,coal_E5,coal_F6,coal_G7,coal_I9,coal_K10,coal_L11,window_id,pos_in_window
0,14,3,1,2019-07-18,41129.4515,40924.6355,204.816,204.816,30.384217,22.5,25.1,23.837500,64.0,84.0,72.625000,0.0,0.1,0.004167,3_1,0,7.0,0.011109,0.043937,0.135335,0.324652,0.606531,0.882497,1.0,0.882497,0.606531,0.324652,0.135335,0.043937,1,0,0,0,0,0,0,0,0,0,0
1,15,3,1,2019-07-19,17861.4665,17861.4665,0.000,204.816,21.462247,23.0,24.9,24.033333,51.0,77.0,65.166667,0.0,0.0,0.000000,3_1,0,7.0,0.011109,0.043937,0.135335,0.324652,0.606531,0.882497,1.0,0.882497,0.606531,0.324652,0.135335,0.043937,1,0,0,0,0,0,0,0,0,0,1
2,16,3,1,2019-07-20,24048.4975,24048.4975,0.000,204.816,45.019395,22.8,25.4,24.033333,55.0,77.0,66.625000,0.0,0.0,0.000000,3_1,0,7.0,0.011109,0.043937,0.135335,0.324652,0.606531,0.882497,1.0,0.882497,0.606531,0.324652,0.135335,0.043937,1,0,0,0,0,0,0,0,0,0,2
3,17,3,1,2019-07-21,749.8105,749.8105,0.000,204.816,27.090391,23.2,26.4,24.633333,56.0,78.0,66.375000,0.0,0.5,0.029167,3_1,0,7.0,0.011109,0.043937,0.135335,0.324652,0.606531,0.882497,1.0,0.882497,0.606531,0.324652,0.135335,0.043937,1,0,0,0,0,0,0,0,0,0,3
4,18,3,1,2019-07-22,199.4695,199.4695,0.000,204.816,31.981250,21.2,26.0,23.866667,61.0,84.0,71.208333,0.0,0.0,0.000000,3_1,0,7.0,0.011109,0.043937,0.135335,0.324652,0.606531,0.882497,1.0,0.882497,0.606531,0.324652,0.135335,0.043937,1,0,0,0,0,0,0,0,0,0,4


In [158]:
print(frames[["stack_id", "date", "window_id", "pos_in_window"]].head(20))

   stack_id       date  window_id  pos_in_window
0       3_1 2019-07-18          0              0
1       3_1 2019-07-19          0              1
2       3_1 2019-07-20          0              2
3       3_1 2019-07-21          0              3
4       3_1 2019-07-22          0              4
5       3_1 2019-07-23          0              5
6      3_10 2019-05-21          1              0
7      3_10 2019-05-22          1              1
8      3_10 2019-05-23          1              2
9      3_10 2019-05-24          1              3
10     3_10 2019-05-25          1              4
11     3_10 2019-05-26          1              5
12     3_10 2019-05-24          2              0
13     3_10 2019-05-25          2              1
14     3_10 2019-05-26          2              2
15     3_10 2019-05-27          2              3
16     3_10 2019-05-28          2              4
17     3_10 2019-05-29          2              5
18     3_10 2019-05-27          3              0
19     3_10 2019-05-

In [159]:
frames.shape

(2664, 44)

In [160]:
len(frames['window_id'].unique())

444

In [161]:
def visualize_frames_timeline(frames: pd.DataFrame, sample_size=100, random_state=42):

    ranges = (
        frames.groupby(["window_id", "stack_id"])
        .agg(start_date=("date", "min"), end_date=("date", "max"))
        .reset_index()
    )

    if len(ranges) > sample_size:
        ranges = ranges.sample(sample_size, random_state=random_state)

    fig = px.timeline(
        ranges,
        x_start="start_date",
        x_end="end_date",
        y="window_id",
        color="stack_id",
        title="Окна временных рядов по stack_id",
        
    )
    fig.update_traces(width=5)
    fig.update_yaxes(title_text="window_id (окна)")
    fig.update_xaxes(title_text="Дата")
    fig.update_layout(
        height=500,
        width=800,
        showlegend=False,
        bargap=0.01,
        xaxis=dict(showgrid=True),
        yaxis=dict(showgrid=False),
    )

    fig.show()

In [162]:
visualize_frames_timeline(frames, sample_size=444)


## Разделение на тестовую и обучающую выборку

In [163]:
def stratified_split_by_windows(
    frames: pd.DataFrame,
    test_size: float = 0.1,
    random_state: int = 42,
    fire_col: str = "fire_flag",
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    frames    : датафрейм после cut_time_series_frames
    test_size : доля окон в тесте (по window_id)
    fire_col  : имя столбца с флагом пожара по дням

    Возвращает:
        frames_train, frames_test, windows_stats
    """

    if "window_id" not in frames.columns:
        raise ValueError("В frames должен быть столбец 'window_id'.")

    if fire_col not in frames.columns:
        raise ValueError(f"В frames нет столбца {fire_col!r}.")

    windows_stats = (
        frames.groupby("window_id")[fire_col]
        .agg(["sum", "count"])
        .reset_index()
        .rename(columns={"sum": "n_fires", "count": "len_window"})
    )

    # страта: есть ли хоть один пожар в окне
    windows_stats["strata"] = (windows_stats["n_fires"] > 0).astype(int)


    train_ids, test_ids = train_test_split(
        windows_stats["window_id"],
        test_size=test_size,
        random_state=random_state,
        stratify=windows_stats["strata"],
    )

    frames_train = frames[frames["window_id"].isin(train_ids)].copy()
    frames_test = frames[frames["window_id"].isin(test_ids)].copy()

    return frames_train, frames_test, windows_stats

In [164]:
frames_train, frames_test, windows_stats = stratified_split_by_windows(
    frames,
    test_size=0.1,
    random_state=42,
    fire_col="fire_flag",
)


In [165]:
print("train windows:", frames_train["window_id"].nunique())
print("test  windows:", frames_test["window_id"].nunique())

print("fires share train:", frames_train["fire_flag"].mean())
print("fires share test :", frames_test["fire_flag"].mean())
print("fires share full :", frames["fire_flag"].mean())

train windows: 399
test  windows: 45
fires share train: 0.03592314118629908
fires share test : 0.037037037037037035
fires share full : 0.036036036036036036


In [166]:
frames_train.columns

Index(['orig_index', 'depot', 'stack', 'date', 'daily_input', 'daily_output',
       'daily_delta', 'accum_weight', 'stack_temp_mean', 'temp_air_min',
       'temp_air_max', 'temp_air_mean', 'humidity_min', 'humidity_max',
       'humidity_mean', 'precipitation_min', 'precipitation_max',
       'precipitation_mean', 'stack_id', 'fire_flag', 'month', 'rbf_month_01',
       'rbf_month_02', 'rbf_month_03', 'rbf_month_04', 'rbf_month_05',
       'rbf_month_06', 'rbf_month_07', 'rbf_month_08', 'rbf_month_09',
       'rbf_month_10', 'rbf_month_11', 'rbf_month_12', 'coal_A1', 'coal_B2',
       'coal_C3', 'coal_E5', 'coal_F6', 'coal_G7', 'coal_I9', 'coal_K10',
       'coal_L11', 'window_id', 'pos_in_window'],
      dtype='object')

In [173]:
temporary_features = ['daily_input',
       'daily_output', 'daily_delta', 'accum_weight', 'stack_temp_mean',
       'temp_air_min', 'temp_air_max', 'temp_air_mean', 'humidity_min',
       'humidity_max', 'humidity_mean', 'precipitation_min',
       'precipitation_max', 'precipitation_mean']
additional_features = ['rbf_month_01', 'rbf_month_02', 'rbf_month_03', 'rbf_month_04',
       'rbf_month_05', 'rbf_month_06', 'rbf_month_07', 'rbf_month_08',
       'rbf_month_09', 'rbf_month_10', 'rbf_month_11', 'rbf_month_12','coal_A1', 'coal_B2',
       'coal_C3', 'coal_E5', 'coal_F6', 'coal_G7', 'coal_I9', 'coal_K10',
       'coal_L11',]

In [ ]:
from tsfresh.feature_extraction.feature_calculators import (
    mean, standard_deviation, maximum, minimum,
    skewness, kurtosis,
    absolute_sum_of_changes,
    autocorrelation,
)

fc_params = {
    "mean": None,
    "standard_deviation": None,
    "maximum": None,
    "minimum": None,
    "skewness": None,
    "kurtosis": None,
    "absolute_sum_of_changes": None,
    "autocorrelation": [{"lag": 1}, {"lag": 2}, {"lag": 3}],
}

In [179]:
def build_tsfresh_dataset(
    frames,
    temporary_features,
    additional_features,
    temp_col="stack_temp_mean",
    id_col="window_id",
    time_col="date",
):
    """
    frames            : результат cut_time_series_frames
                        (должны быть столбцы window_id, pos_in_window, date, stack_id, depot, stack, temp_col)
    temporary_features: список временных признаков, из которых извлекаем tsfresh-фичи
    additional_features: список "статических" признаков (rbf_month_*, coal_*)
                         которые просто подтягиваем из последней строки окна
    temp_col          : название колонки с температурой штабеля (таргет)
    """

    required = {id_col, "pos_in_window", time_col, "stack_id", "depot", "stack", temp_col}
    missing = required - set(frames.columns)
    if missing:
        raise ValueError(f"В frames отсутствуют столбцы: {missing}")

    df = frames.copy()

    # история = все строки, кроме последней по pos_in_window в каждом окне
    max_pos = df.groupby(id_col)["pos_in_window"].transform("max")
    hist = df[df["pos_in_window"] < max_pos].copy()

    if hist.empty:
        raise ValueError("В истории нет строк (hist пустой). Проверьте pos_in_window / окна.")

    # tsfresh не нужен по самой дате, используем её только как time_col
    value_cols = [c for c in temporary_features if c != time_col]

    # данные для tsfresh
    ts_df = hist[[id_col, time_col] + value_cols].sort_values([id_col, time_col])

    fc_params = {
        "mean": None,
        "standard_deviation": None,
        "maximum": None,
        "minimum": None,
        "skewness": None,
        "kurtosis": None,
        "absolute_sum_of_changes": None,
        "autocorrelation": [{"lag": 1}, {"lag": 2}, {"lag": 3}],
}

    ts_features = extract_features(
        ts_df,
        column_id=id_col,
        column_sort=time_col,
        default_fc_parameters=fc_params,
        disable_progressbar=True,
    )

    # последняя строка окна = "целевой день"
    last_rows = (
        df.sort_values(time_col)
        .groupby(id_col)
        .tail(1)
        .set_index(id_col)
    )

    cols_to_keep = ["stack_id", "depot", "stack", time_col, temp_col] + additional_features
    cols_to_keep = [c for c in cols_to_keep if c in last_rows.columns]

    meta = last_rows[cols_to_keep].copy()
    meta = meta.rename(
        columns={
            time_col: "target_date",
            temp_col: "target_temp",
        }
    )

    # объединяем мета-инфу и tsfresh-фичи по window_id
    result = meta.join(ts_features, how="left")
    result = result.reset_index()  # вернём window_id в явный столбец

    return result

In [180]:
frames_train_add = build_tsfresh_dataset(
    frames=frames_train,
    temporary_features=temporary_features,
    additional_features=additional_features,
    temp_col="stack_temp_mean",
    id_col="window_id",
    time_col="date",
)

print(frames_train_add.shape)
print(frames_train_add.head())


(399, 167)
   window_id stack_id  depot  stack target_date  target_temp  rbf_month_01  \
0        117     3_37      3     37  2019-01-07    66.630883           1.0   
1        246     4_28      4     28  2019-01-07    76.256826           1.0   
2         40     3_21      3     21  2019-01-08    25.021170           1.0   
3         41     3_22      3     22  2019-01-09    35.799361           1.0   
4        247     4_28      4     28  2019-01-10    33.676990           1.0   

   rbf_month_02  rbf_month_03  rbf_month_04  rbf_month_05  rbf_month_06  \
0      0.882497      0.606531      0.324652      0.135335      0.043937   
1      0.882497      0.606531      0.324652      0.135335      0.043937   
2      0.882497      0.606531      0.324652      0.135335      0.043937   
3      0.882497      0.606531      0.324652      0.135335      0.043937   
4      0.882497      0.606531      0.324652      0.135335      0.043937   

   rbf_month_07  rbf_month_08  rbf_month_09  rbf_month_10  rbf_month_

In [181]:
frames_train_add.columns

Index(['window_id', 'stack_id', 'depot', 'stack', 'target_date', 'target_temp',
       'rbf_month_01', 'rbf_month_02', 'rbf_month_03', 'rbf_month_04',
       ...
       'precipitation_mean__mean', 'precipitation_mean__standard_deviation',
       'precipitation_mean__maximum', 'precipitation_mean__minimum',
       'precipitation_mean__skewness', 'precipitation_mean__kurtosis',
       'precipitation_mean__absolute_sum_of_changes',
       'precipitation_mean__autocorrelation__lag_1',
       'precipitation_mean__autocorrelation__lag_2',
       'precipitation_mean__autocorrelation__lag_3'],
      dtype='object', length=167)

In [182]:
frames_test_add = build_tsfresh_dataset(
    frames=frames_test,
    temporary_features=temporary_features,
    additional_features=additional_features,
    temp_col="stack_temp_mean",
    id_col="window_id",
    time_col="date",
)

print(frames_test_add.shape)
print(frames_test_add.head())

(45, 167)
   window_id stack_id  depot  stack target_date  target_temp  rbf_month_01  \
0        161     3_52      3     52  2019-05-11    34.962361      0.135335   
1        248     4_28      4     28  2019-05-13    28.998402      0.135335   
2        175      3_9      3      9  2019-05-24    33.756413      0.135335   
3        122      3_4      3      4  2019-06-01    28.643588      0.043937   
4        218     4_16      4     16  2019-06-03    23.064420      0.043937   

   rbf_month_02  rbf_month_03  rbf_month_04  rbf_month_05  rbf_month_06  \
0      0.324652      0.606531      0.882497      1.000000      0.882497   
1      0.324652      0.606531      0.882497      1.000000      0.882497   
2      0.324652      0.606531      0.882497      1.000000      0.882497   
3      0.135335      0.324652      0.606531      0.882497      1.000000   
4      0.135335      0.324652      0.606531      0.882497      1.000000   

   rbf_month_07  rbf_month_08  rbf_month_09  rbf_month_10  rbf_month_1

In [183]:
frames_train_add.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Columns: 167 entries, window_id to precipitation_mean__autocorrelation__lag_3
dtypes: datetime64[ns](1), float64(153), int64(12), object(1)
memory usage: 520.7+ KB


## Обучение baseline модели

In [184]:
target_col = "target_temp"

base_drop_cols = [
    target_col,
    "target_date",
    "start_date",
    "end_date",
    "window_id",
    "stack_id",
    "depot",
    "stack",
]

# всё, что не должно идти в признаки
leak_cols = ["date","fire_flag"]

drop_cols_train = [c for c in base_drop_cols + leak_cols if c in frames_train_add.columns]
drop_cols_test = [c for c in base_drop_cols + leak_cols if c in frames_test_add.columns]

num_cols_train = frames_train_add.select_dtypes(include=["number"]).columns.tolist()
feature_cols = [c for c in num_cols_train if c not in drop_cols_train]

X_train = frames_train_add[feature_cols].copy()
y_train = frames_train_add[target_col].astype(float).copy()

X_test = frames_test_add.reindex(columns=feature_cols, fill_value=0).copy()
y_test = frames_test_add[target_col].astype(float).copy()

print("Train shape:", X_train.shape)
print("Test  shape:", X_test.shape)
print("Features count:", len(feature_cols))


Train shape: (399, 161)
Test  shape: (45, 161)
Features count: 161


In [ ]:

pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("model", RandomForestRegressor(random_state=42, n_jobs=-1)),
    ]
)

param_distributions = {
    "model__n_estimators": [100, 200, 400, 600],
    "model__max_depth": [3, 5, 7, 9, None],
    "model__min_samples_split": [2, 4, 8, 16],
    "model__min_samples_leaf": [1, 2, 4, 8],
    "model__max_features": ["sqrt", "log2", 0.3, 0.5, 0.8],
}

search = RandomizedSearchCV(
    estimator=pipe,
    param_distributions=param_distributions,
    n_iter=20,
    scoring="neg_mean_absolute_error",
    cv=3,
    n_jobs=-1,
    verbose=1,
    random_state=42,
)

search.fit(X_train, y_train)

print("Лучшие параметры:")
print(search.best_params_)
print("\nЛучшая MAE на CV:", -search.best_score_)

best_model = search.best_estimator_




Fitting 3 folds for each of 20 candidates, totalling 60 fits


/home/redac/miniconda3/envs/torch12/lib/python3.10/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['precipitation_min__autocorrelation__lag_1'
 'precipitation_min__autocorrelation__lag_2'
 'precipitation_min__autocorrelation__lag_3']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/home/redac/miniconda3/envs/torch12/lib/python3.10/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['precipitation_min__autocorrelation__lag_1'
 'precipitation_min__autocorrelation__lag_2'
 'precipitation_min__autocorrelation__lag_3']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(
/home/redac/miniconda3/envs/torch12/lib/python3.10/site-packages/sklearn/impute/_base.py:653: UserWarning: Skipping features without any observed values: ['precipitation_min__autocorrelation__lag_1'
 'precipitation_min__auto

Лучшие параметры:
{'model__n_estimators': 200, 'model__min_samples_split': 16, 'model__min_samples_leaf': 4, 'model__max_features': 0.5, 'model__max_depth': 3}

Лучшая MAE на CV: 9.447157959911442


In [187]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

y_pred = best_model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"\nMAE = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")
print(f"R2 = {r2:.4f}")


MAE = 8.1013
RMSE = 13.7319
R2 = 0.6114


In [242]:
best_rf = search.best_estimator_

In [243]:
import pickle

model_path = "rf_best_pipeline.pkl"  

with open(model_path, "wb") as f:
    pickle.dump(best_rf, f)

print(f"Модель сохранена в {model_path}")


Модель сохранена в rf_best_pipeline.pkl


In [197]:
# 1. Бинарный флаг "был ли пожар в этом окне вообще"
fire_in_window = (
    frames_test
    .groupby("window_id")["fire_flag"]
    .max()                     # если в окне есть хоть один 1, будет 1
    .astype(int)
)

# 2. Добавляем в frames_test_add
frames_test_add = frames_test_add.merge(
    fire_in_window.rename("fire_in_window"),
    how="left",
    left_on="window_id",
    right_index=True,
)

print("fire_in_window" in frames_test_add.columns)
print(frames_test_add["fire_in_window"].value_counts())

True
fire_in_window
0    42
1     3
Name: count, dtype: int64


In [ ]:
y_pred = best_model.predict(X_test)

CRITICAL_TEMP = 50.0  

# истинные метки: был ли пожар в этом окне (хотя бы один день)
y_true_fire = frames_test_add.loc[X_test.index, "fire_in_window"].astype(int)

# непрерывный скор для ROC AUC — предсказанная температура
y_score = y_pred

# бинарный прогноз по порогу
y_pred_fire = (y_score > CRITICAL_TEMP).astype(int)

roc_auc = roc_auc_score(y_true_fire, y_score)

print(f"Критическая температура: {CRITICAL_TEMP}")
print(f"ROC AUC = {roc_auc:.4f}")
print(f"Доля предсказанных окон с пожаром: {y_pred_fire.mean():.4f}")
print(f"Доля реальных окон с пожаром: {y_true_fire.mean():.4f}")


Критическая температура: 50.0
ROC AUC = 0.8968
Доля предсказанных окон с пожаром: 0.0444
Доля реальных окон с пожаром: 0.0667


## Обучение бустинга через optuna

In [232]:
def objective(trial):

    n_estimators = trial.suggest_int("n_estimators", 100, 600)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.2, log=True)
    max_depth = trial.suggest_int("max_depth", 2, 5)
    subsample = trial.suggest_float("subsample", 0.6, 1.0)
    max_features = trial.suggest_categorical("max_features", ["sqrt", "log2", None])

    model = GradientBoostingRegressor(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample,
        max_features=max_features,
        random_state=42,
    )

    pipe = Pipeline(
        steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", StandardScaler()),
            ("model", model),
        ]
    )


    tscv = TimeSeriesSplit(n_splits=3)

    scores = cross_val_score(
        pipe,
        X_train,
        y_train,
        cv=tscv,
        scoring="neg_mean_absolute_error",
        n_jobs=-1,
    )

    return -scores.mean()


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=300, show_progress_bar=False)

print("Лучшие параметры бустинга:")
print(study.best_params)
print("Лучшая MAE на CV:", study.best_value)

[I 2025-11-24 12:50:54,384] A new study created in memory with name: no-name-9ca3d19f-849d-42f0-b75b-bc8d38a703fc
[I 2025-11-24 12:50:54,469] Trial 0 finished with value: 12.160893103640838 and parameters: {'n_estimators': 179, 'learning_rate': 0.12612277166438457, 'max_depth': 2, 'subsample': 0.8135994125442249, 'max_features': 'log2'}. Best is trial 0 with value: 12.160893103640838.
[I 2025-11-24 12:50:56,695] Trial 1 finished with value: 11.092565786010526 and parameters: {'n_estimators': 360, 'learning_rate': 0.09422807806905456, 'max_depth': 4, 'subsample': 0.8285884950448225, 'max_features': None}. Best is trial 1 with value: 11.092565786010526.
[I 2025-11-24 12:51:00,482] Trial 2 finished with value: 10.887750337461867 and parameters: {'n_estimators': 512, 'learning_rate': 0.01098606803981784, 'max_depth': 2, 'subsample': 0.7878251673840595, 'max_features': None}. Best is trial 2 with value: 10.887750337461867.
[I 2025-11-24 12:51:00,556] Trial 3 finished with value: 12.52708369

Лучшие параметры бустинга:
{'n_estimators': 108, 'learning_rate': 0.013089988279829436, 'max_depth': 4, 'subsample': 0.8876649789454097, 'max_features': None}
Лучшая MAE на CV: 10.26701369080303


In [233]:
best_params = study.best_params

best_model = GradientBoostingRegressor(
    n_estimators=best_params["n_estimators"],
    learning_rate=best_params["learning_rate"],
    max_depth=best_params["max_depth"],
    subsample=best_params["subsample"],
    max_features=best_params["max_features"],
    random_state=42,
)

final_pipe = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("model", best_model),
    ]
)

final_pipe.fit(X_train, y_train)

y_pred = final_pipe.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"\nMAE = {mae:.4f}")
print(f"RMSE = {rmse:.4f}")
print(f"R2 = {r2:.4f}")


MAE = 9.4563
RMSE = 15.7484
R2 = 0.4889


In [236]:
# 1. Бинарный флаг "был ли пожар в этом окне вообще"
fire_in_window = (
    frames_test
    .groupby("window_id")["fire_flag"]
    .max()                     # если в окне есть хоть один 1, будет 1
    .astype(int)
)

# 2. Добавляем в frames_test_add
frames_test_add = frames_test_add.merge(
    fire_in_window.rename("fire_in_window"),
    how="left",
    left_on="window_id",
    right_index=True,
)

print("fire_in_window" in frames_test_add.columns)
print(frames_test_add["fire_in_window"].value_counts())

True
fire_in_window
0    42
1     3
Name: count, dtype: int64


In [239]:
y_pred = final_pipe.predict(X_test)

CRITICAL_TEMP = 50.0  

# истинные метки: был ли пожар в этом окне (хотя бы один день)
y_true_fire = frames_test_add.loc[X_test.index, "fire_in_window"].astype(int)

# непрерывный скор для ROC AUC — предсказанная температура
y_score = y_pred

# бинарный прогноз по порогу
y_pred_fire = (y_score > CRITICAL_TEMP).astype(int)

roc_auc = roc_auc_score(y_true_fire, y_score)

print(f"Критическая температура: {CRITICAL_TEMP}")
print(f"ROC AUC = {roc_auc:.4f}")
print(f"Доля предсказанных окон с пожаром: {y_pred_fire.mean():.4f}")
print(f"Доля реальных окон с пожаром: {y_true_fire.mean():.4f}")


Критическая температура: 50.0
ROC AUC = 0.9206
Доля предсказанных окон с пожаром: 0.0889
Доля реальных окон с пожаром: 0.0667


In [244]:
model_path = "gdr_best_pipeline.pkl"  

with open(model_path, "wb") as f:
    pickle.dump(final_pipe, f)

print(f"Модель сохранена в {model_path}")

Модель сохранена в gdr_best_pipeline.pkl
